pip install numpy xarray geoviews cartopy hvplot jupyter_bokeh  
Zarr와 fsspec의 모든 원격 기능을 활성화하는 설치 방식  
pip install "zarr[remote]" "fsspec[http]"

### 1. 데이터 생성 및 저장

In [3]:
import zarr
import numpy as np

# 1. Zarr 배열 생성 (문자열 경로만 넣으면 자동으로 DirectoryStore로 처리됩니다)
# v3에서는 스토어를 직접 생성하기보다 단순 경로를 선호합니다.
store_path = 'example.zarr'

# 2. 그룹 생성 (zarr.group 대신 zarr.open_group 권장)
root = zarr.open_group(store=store_path, mode='w')

# 3. 데이터셋 생성
foo = root.create_array('data', shape=(1000, 1000), chunks=(100, 100), dtype='f4')

# 4. 데이터 쓰기
foo[:] = np.random.standard_normal(size=(1000, 1000))

print(f"저장 완료: {store_path}")
print(root.info)

저장 완료: example.zarr
Name        : 
Type        : Group
Zarr format : 3
Read-only   : False
Store type  : LocalStore


### 2.무료 예제 데이터 로딩 및 지도시각화

In [7]:
import xarray as xr
import hvplot.xarray

# 1. 공공 S3 버킷의 Zarr 데이터 로드 (분산 저장된 데이터를 즉시 연결)
# 예: 실시간 구름 또는 해수면 온도 데이터
url = "https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1"
ds = xr.open_zarr(url, consolidated=True)

# 2. 특정 영역 및 시간대 슬라이싱 (실제 데이터는 이때 필요한 만큼만 다운로드됨)
subset = ds.analysed_sst.sel(time='2020-01-01', lat=slice(20, 50), lon=slice(120, 150))

# 3. 지도시각화
subset.hvplot.image(geo=True, cmap='viridis', title="SST near Korea (2020-01-01)")

BokehModel(combine_events=True, render_bundle={'docs_json': {'6331a58a-9c3e-4438-941b-590f9304cb61': {'version…

### 3.포맷 변환 (NetCDF → Zarr)

In [8]:
# NetCDF 읽어서 Zarr로 저장
ds_nc = xr.open_dataset('ds_kor2.nc')
ds_nc.to_zarr('converted_data.zarr', mode='w', consolidated=True)

c:\Users\thlee\AppData\Local\Programs\Python\Python311\Lib\site-packages\zarr\api\asynchronous.py:247: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


### 4.주요 리소스

공식 문서 (Zarr Python): https://zarr.readthedocs.io/

Zarr 사양(Spec) 정보: https://zarr-specs.readthedocs.io/

OGC Zarr 표준 페이지: https://www.ogc.org/standard/zarr/

Pangeo Forge (대규모 기상 데이터 변환 프로젝트): https://pangeo-forge.org/